In [45]:
!pip3 install spacy
!python3 -m spacy download fr_core_news_sm
import nltk
nltk.download("punkt")
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import pandas as pd
import networkx as nx
import spacy
import fr_core_news_sm
from sklearn.metrics.pairwise import cosine_similarity
nltk.download("stopwords")
import networkx as nx
from nltk.stem.snowball import SnowballStemmer

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
import os
import requests
from bs4 import BeautifulSoup

requete = requests.get("https://www.lemonde.fr/")
page = requete.content
soupAccueil = BeautifulSoup(page)

# cas d'article live
live = False
if(soupAccueil.find("div", {"class": "article article--main"}).find("span", {"class": "icon__label-live icon__label-live--xl"})):
    live = True
print(live)

False


In [51]:
if(not live):
    mainArticle = {
        "title": soupAccueil.find("div", {"class": "article article--main"}).find("span", {"class": "article__title-label"}).get_text().replace(u'\xa0', u' '),
        "link": soupAccueil.find("div", {"class": "article article--main"}).a["href"]
    }

    requete = requests.get(mainArticle["link"])
    page = requete.content
    soupMainArticle = BeautifulSoup(page)
    mainArticle["subTitle"] = soupMainArticle.find("p", {"class": "article__desc"}).get_text().replace(u'\xa0', u' ')
    paragraphes = soupMainArticle.find_all("p", {"class": "article__paragraph"})
    mainArticle["content"] = list(map(lambda para: para.get_text().replace(u'\xa0', u' '), paragraphes))


    print(mainArticle)

{'title': 'Le gouvernement doute que l’objectif de 5 000 cas de Covid-19 par jour soit atteint le 15 décembre pour enclencher le déconfinement', 'link': 'https://www.lemonde.fr/societe/article/2020/12/07/covid-19-l-executif-pessimiste-sur-l-objectif-des-5-000-cas-par-jour-fixe-pour-enclencher-le-deconfinement_6062505_3224.html', 'subTitle': '    Les nouveaux cas comptabilisés ont cessé de reculer depuis une semaine et se sont stabilisés autour de 10 000 par jour – loin de l’objectif fixé. Le directeur général de la santé, Jérôme Salomon, doit livrer un point de situation ce lundi à 18 heures. ', 'content': ['« On ne sera pas à l’objectif au 15 décembre » : c’est l’analyse qu’ont faite, lundi 7 décembre, le premier ministre, Jean Castex, et le ministre de la santé, Olivier Véran, lors d’une réunion avec les chefs des groupes parlementaires, dans le cadre du Comité de contrôle et de liaison Covid-19. Fin novembre, Emmanuel Macron avait fixé comme condition nécessaire pour lever le confin

In [67]:
#Récupère article sur un lien
live = True
if(live):
    requete = requests.get("https://www.lemonde.fr/sciences/article/2020/12/01/le-telescope-geant-d-arecibo-s-est-effondre_6061835_1650684.html")
    page = requete.content
    soupAccueil = BeautifulSoup(page)
    
    mainArticle = {
        "title": soupAccueil.find("h1", {"class": "article__title"}).get_text().replace(u'\xa0', u' '),
        "link": "https://www.lemonde.fr/sciences/article/2020/12/01/le-telescope-geant-d-arecibo-s-est-effondre_6061835_1650684.html"
    }

    requete = requests.get(mainArticle["link"])
    page = requete.content
    soupMainArticle = BeautifulSoup(page)
    mainArticle["subTitle"] = soupMainArticle.find("p", {"class": "article__desc"}).get_text().replace(u'\xa0', u' ')
    paragraphes = soupMainArticle.find_all("p", {"class": "article__paragraph"})
    mainArticle["content"] = list(map(lambda para: para.get_text().replace(u'\xa0', u' '), paragraphes))

    print(mainArticle)

{'title': 'Le télescope géant d’Arecibo s’est effondré', 'link': 'https://www.lemonde.fr/sciences/article/2020/12/01/le-telescope-geant-d-arecibo-s-est-effondre_6061835_1650684.html', 'subTitle': '    Jugé trop instable et non réparable après la rupture de deux câbles en novembre, le télescope s’est effondré soudainement le 1er décembre. ', 'content': ['C’était l’un des télescopes les plus célèbres de la planète. Sur l’île américaine de Porto Rico, le radiotélescope d’Arecibo, utilisé par les astronomes du monde entier et qui avait permis de découvrir les premières planètes en orbite autour d’une autre étoile que le Soleil, s’est effondré mardi 1er décembre après cinquante-sept ans de service. « La plate-forme s’est écroulée de manière non planifiée », a déclaré à l’Agence France-Presse (AFP) Robert James Margetta, porte-parole de la Fondation nationale des sciences, l’agence américaine qui finance l’observatoire.', 'Deux des câbles soutenant les 900 tonnes d’instruments du télescope s

In [16]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(top_n=5):
 
    stop_words = stopwords.words('french')
    summarize_text = []

    nlp = fr_core_news_sm.load()
    
    # Step 1 - Read text and tokenize
    text = " ".join(mainArticle["content"])
    doc = nlp(text)
    sentences =  [X.text for X in doc.sents]

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    # print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(ranked_sentence[i][1])

    # Step 5 - Offcourse, output the summarize texr
    print("Summarize Text: \n", ".. ".join(summarize_text))

# let's begin
generate_summary(5)

Summarize Text: 
 « Les plus pauvres louent même parfois leurs enfants à une autre famille », explique Tesfaye Seyoum, chargé de formation dans l’établissement.. « Ensuite, ils passent un examen et on les forme », ajoute Abel Kassahun.. Dans les prochaines années, nous prévoyons de former plus de 700 enseignants employés par le gouvernement et près de 300 responsables du secteur de l’éducation, ce qui nous permettra d’atteindre des milliers d’enfants supplémentaires », se félicite Caitlin Baron, la directrice de Luminos Fund.. « Lors de cette rupture, l’école est devenue bien plus inclusive », résume Alemayehu Hailu Gebre, le directeur de la Luminos Fund en Ethiopie, qui gère chaque année 10 000 élèves en rattrapage scolaire accéléré dans le pays et se réjouit que seulement 2 % d’entre eux ont décroché depuis le début de la pandémie due au coronavirus.. A la veille du confinement, lors de la visite du Monde Afrique, les élèves de la classe de Duber, répartis en cinq groupes, apprennent

In [68]:
#-------------------------------- Nouvelle technique ---------------------------------

#tokeniser les mots
def return_token(sentence):
    nlp = fr_core_news_sm.load()
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner le texte de chaque token
    return [X.text for X in doc]

def return_mean_embedding(sentence):
    nlp = fr_core_news_sm.load()
    # Tokeniser la phrase
    doc = nlp(sentence)
    # Retourner la moyenne des vecteurs pour chaque phrase
    return np.mean([(X.vector) for X in doc], axis=0)
 
def build_similarity_matrix(sentences, sentence_vectors):
    # Create an empty similarity matrix
    sim_mat = np.zeros((len(sentences), len(sentences)))
 
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j: #ignore if both are same sentences
                continue 
            sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,96), sentence_vectors[j].reshape(1,96))[0,0]

    return sim_mat


stop_words = stopwords.words('french')
summarize_text = []
nlp = fr_core_news_sm.load()

# Step 1 - Read text and tokenize
text = " ".join(mainArticle["content"])
tokenizer = nltk.data.load('tokenizers/punkt/PY3/french.pickle')
phrases = tokenizer.tokenize(text)

# Preprocessing
# Filtre les stop words et stemming
stemmer = SnowballStemmer(language='french')
clean_sentences = []
order = []
idx = 0
for sentence in phrases:
  clean_words = []
  order.append(i)
  i += 1
  for token in return_token(sentence):
    if token not in stop_words:
      clean_words.append(stemmer.stem(token))
  clean_sentences.append(" ".join(clean_words))

# New steps : vectorisation de nos phrases
phrases_vector = []
for sentence in clean_sentences:
  phrases_vector.append(return_mean_embedding(sentence))
# Step 2 - Generate Similary Martix across sentences
sentence_similarity_martix = build_similarity_matrix(phrases, phrases_vector)

# Step 3 - Rank sentences in similarity martix
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
scores = nx.pagerank(sentence_similarity_graph)
# Step 4 - Sort the rank and pick top sentences
ranked_sentence = sorted(((scores[i],s,order[i]) for i,s in enumerate(phrases)), reverse=True)
print(ranked_sentence)

top_sents = []
for i in range(5):
  top_sents.append((ranked_sentence[i][1], ranked_sentence[i][2]))

summarize_text = sorted(top_sents, key=lambda ordre: ordre[1])
print(top_sents)
print(summarize_text)
# Step 5 - Offcourse, output the summarize text
print("Summarize Text: \n", ".. ".join((summarize_text[i][0] for i in range(5))))

[(0.04, '« Nombre d’étudiants se formaient à l’astronomie dans l’observatoire, c’est ce qui leur donne l’inspiration de faire une carrière en sciences ou en astronomie, comme moi », a-t-il ajouté.', 13), (0.04, '« Même de Chine », a précisé Abel Méndez.', 15), (0.04, '« La plate-forme s’est écroulée de manière non planifiée », a déclaré à l’Agence France-Presse (AFP) Robert James Margetta, porte-parole de la Fondation nationale des sciences, l’agence américaine qui finance l’observatoire.', 6), (0.04, '« La perte d’Arecibo est une grande perte pour le monde, mais encore plus pour Porto Rico.', 21), (0.04, '« C’est un désastre absolu », a réagi, ému, le professeur Abel Méndez, directeur du laboratoire habitabilité planétaire de l’université de Porto Rico, auprès de l’AFP.', 12), (0.04, 'Une scène d’action du film GoldenEye s’était également déroulée au-dessus du télescope.', 24), (0.04, 'Tous les astronomes de la planète pouvaient demander une portion de temps du radiotélescope pour fai